In [ ]:
# ------------------------------------------------------------
# STEP 6B — U-Net (R34, ImageNet encoder) — PAPER BASELINE
# Cell 1/4 — Imports & tiny wrapper to keep API consistent
# ------------------------------------------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp

class SMPWrap(nn.Module):
    """Wrap an SMP model so forward(x) -> {'logits': (B,1,H,W)}"""
    def __init__(self, core: nn.Module):
        super().__init__()
        self.core = core
    def forward(self, x):
        logits = self.core(x)              # raw logits from SMP (activation=None)
        if isinstance(logits, (list, tuple)):
            logits = logits[0]
        return {"logits": logits}


In [ ]:
# ------------------------------------------------------------
# STEP 6B — U-Net definition (R34 encoder, classic decoder)
# ------------------------------------------------------------
# Cell 2/4
# ------------------------------------------------------------
def build_unet_r34_binary():
    model = smp.Unet(
        encoder_name="resnet34",           # paper-friendly backbone
        encoder_weights="imagenet",        # pretrained encoder
        in_channels=3,
        classes=1,
        activation=None                    # raw logits; sigmoid in loss
    )
    return SMPWrap(model)


In [ ]:
# ------------------------------------------------------------
# STEP 6B — Instantiate & quick shape check
# ------------------------------------------------------------
# Cell 3/4
# ------------------------------------------------------------
unet_r34_model = build_unet_r34_binary()
xb, yb = next(iter(train_loader))
out = unet_r34_model(xb)
print("U-Net (R34) logits:", tuple(out["logits"].shape))


In [ ]:
# ------------------------------------------------------------
# STEP 6B — Params
# ------------------------------------------------------------
# Cell 4/4
# ------------------------------------------------------------
num_params = sum(p.numel() for p in unet_r34_model.parameters() if p.requires_grad)
print(f"[PARAMS] U-Net (R34): {num_params/1e6:.2f} M")

